# 2.252 Derivative of the polynomial

*https://stepik.org/lesson/28258/step/1?adaptive=true*

The problem is simple – find the derivative of the polynomial.
The polynomial can be large, but only with
non-negative integral powers and integer coefficients.
Think about how it is better to represent it in the memory?
The output should contain no spaces and be in the descending order of powers,
although the input may not comply with this rule.
Additionally, the input may contain a polynomial with unreduced terms.

In [285]:
from re import sub
from collections import defaultdict

def polynomial_derivative(inp):
    dic = defaultdict(int)
    for t in inp.replace('-', '+-').lstrip('+').split('+'):
        t = sub('^(-?)x', r'\g<1>1*x', sub('x$', 'x^1', sub(r'^(-?\d+)$', r'\1*x^0', t)))
        c, p = map(int, t.split('*x^'))
        dic[p-1] += c*p
    lst = [sub(r'\*x\^1$', '*x', sub(r'\*x\^0$', '', '%d*x^%d' % (c,p)))
           for p, c in sorted(dic.items(), reverse=True) if c]
    return '+'.join(lst).replace('+-', '-') or '0'

test = {
    'x^2+x': '2*x+1',
    '2*x^100+100*x^2': '200*x^99+200*x',
    'x^10000+x+1': '10000*x^9999+1',
    '-x^2-x^3': '-3*x^2-2*x',
    'x+x+x+x+x+x+x+x+x+x': '10',
}
for inp, out in test.items():
    print(inp, '->', polynomial_derivative(inp))
    assert polynomial_derivative(inp) == out

x^2+x -> 2*x+1
2*x^100+100*x^2 -> 200*x^99+200*x
x^10000+x+1 -> 10000*x^9999+1
-x^2-x^3 -> -3*x^2-2*x
x+x+x+x+x+x+x+x+x+x -> 10


# 2.168 List of prime numbers

*https://stepik.org/lesson/28173/step/1?adaptive=true*

Let's create a list (or array) of prime numbers in ascending order:

    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, ...]

And then let's output the ones that correspond to the indices,
specified in the input data.

Input data in the first line specifies the number of prime numbers to be output.
The second line contains the indices of interest to us in the array of prime numbers.
They will be in the range from 1 to 200000.

The output should contain the prime numbers,
corresponding to the specified positions in the array of all primes.

Please note: in this problem indices of the array start from 1 and not from 0,
so it will be easier to compare with primes lists from the internet.

Sample Input:

    4
    1 3 5 8

Sample Output:

    2 5 11 19

### Метод "в лоб"

In [286]:
from itertools import islice, count

def prime_gen():
    yield 1; yield 2; yield 3; yield 5
    for n in count(7,2):
        if n%3 and n%5:
            for d in range(7, int(n**.5)+1):
                if not n%d:  break
            else:
                yield n

In [287]:
ind = [1,3,5,8]
pri = list(islice(prime_gen(), max(ind)+1))
print(*(pri[i] for i in ind))

2 5 11 19


In [288]:
%timeit -r1 -n1 print(list(islice(prime_gen(), 200000+1))[200000])

2750159
28.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [289]:
cnt = int(input())
ind = list(map(int, input().split()))[:cnt]
pri = list(islice(prime_gen(), max(ind)+1))
print(*(pri[i] for i in ind))

4
1 3 5 8
2 5 11 19


### Решето Эратосфена с ограничением сверху

In [290]:
PRIME_200K = 2750159

def prime_sieve(end=PRIME_200K):
    "Erathosthenes sieve"
    end += 1
    sieve = set(range(1, end))
    for p in range(2, int(end**.5)):
        for i in range(p*p, end, p):
            if i in sieve:
                sieve.remove(i)
    return sorted(sieve)

In [291]:
ind = [1,3,5,8]
pri = prime_sieve()
print(*(pri[i] for i in ind))

2 5 11 19


In [292]:
%timeit -r1 -n1 print(prime_sieve()[200000])

2750159
6.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [293]:
cnt = int(input())
ind = list(map(int, input().split()))[:cnt]
end = 2750159 + 1  # 200000th prime number
pri = set(range(1, end))
for p in range(2, int(end**.5)):
    for i in range(p*p, end, p):
        if i in pri:
            pri.remove(i)
pri = sorted(pri)
print(*(pri[i] for i in ind))

4
1 3 5 8
2 5 11 19


### Комбинированный вариант

Сложность кода повышена в угоду оптимизации. Если у программы запрашивают простые числа в небольшом количестве (меньше 50к), то используется прямолино написанный генератор, который просто проверяет очередного кандидата на делимость. При этом затрачивается минимум оперативки, а общее время выполнения сокращается за счёт того, что генерируется мало чисел. Если же запрашивают простые числа в большом диапазоне, то применяется более оптимальное по времени выполнения (но не по памяти) решето Эратосфена с максимумом, равным 200-тысячному простому числу (из условия задачи).

In [294]:
# combined solution
from itertools import islice, count
PRIME_200K = 2750159
THRESHOLD = 50000

cnt = int(input())
ind = list(map(int, input().split()))[:cnt]
end = max(ind)

if end > THRESHOLD:
    # use Erathosthene`s sieve for large lists of primes
    end = PRIME_200K + 1  # 200000th prime number
    sieve = set(range(1, end))
    for p in range(2, int(end**.5)):
        for i in range(p*p, end, p):
            if i in sieve:
                sieve.remove(i)
    pri = sorted(sieve)
else:
    # simple`n`dirty generator for smaller lists of primes
    def prime_gen():
        yield 1; yield 2; yield 3; yield 5
        for n in count(7,2):
            if n%3 and n%5:
                for d in range(7, int(n**.5)+1):
                    if not n%d:  break
                else:
                    yield n
    pri = list(islice(prime_gen(), end+1))

print(*(pri[i] for i in ind))

4
1 3 5 8
2 5 11 19


### Бесконечный генератор на основе решета Эрастофена

In [318]:
from itertools import islice, count

def prime_sieve_gen():
    yield 1
    s = {}
    for q in count(2):
        p = s.pop(q, None)
        if p is None:
            yield q
            s[q * q] = q
        else:
            x = p + q
            while x in s:  x += p
            s[x] = p

In [322]:
print(list(islice(prime_sieve_gen(), 200000, 200000+1))[0])

2750159


In [323]:
%timeit list(islice(prime_sieve_gen(), 200000, 200000+1))

2.24 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
